In [44]:
import pandas as pd
from datetime import datetime
pd.options.display.float_format = '{:,.2f}'.format


In [45]:
def get_un_data(un_input_path):
    col_pair = {
        'Aggregate Level': 'int8',
        'Trade Flow Code': 'int8',
        'Reporter Code': 'int16',
        'Partner Code': 'int16',
        'Commodity Code': str,
        'Trade Value (US$)': float
    }

    def parse_col_name(v):
        for char in ['(', ')', '.', '$']:
            v = v.replace(char, '')
        return v.replace(' ', '_').lower()

    def select_necessary_data(df):
        # trade_flow = {1: 'import', 2: 'export'}
        q = ('trade_flow_code == 2 & aggregate_level == 6 & '
             'partner_code == 0')
        needless_cols = ['aggregate_level', 'trade_flow_code']
        return df.query(q).drop(needless_cols, axis=1)\
                          .astype({'commodity_code': str})\
                          .reset_index(drop=True)

    df = pd.read_csv(un_input_path, usecols=[k for k in col_pair], dtype=col_pair)
    df.columns = [parse_col_name(c) for c in df.columns]
    return df.pipe(select_necessary_data)

# Load Files 

In [46]:
un_input_path = '/Users/Urien/Desktop/Taitra/1120/type-C_r-ALL_ps-2016_freq-A_px-HS_pub-20170912_fmt-csv_ex-20170928.csv'
country_code = pd.read_excel('./un_country_codes.xls',usecols=['Country Code','Country Name English'])
country_code.columns=['reporter_code','reporter_name']
hs_classes = pd.read_csv('./hscode_v1121.csv', sep=',', encoding='utf-8',dtype={'industry_name': str, 'industry_code': str})

In [10]:
df = get_un_data(un_input_path)

In [49]:
def write_log(_str):
    filename = datetime.now().strftime('%Y%m%d-%H%M%S')
    f = open("./log/log{}.txt".format(filename),"a")
    f.write(str(_str))
    f.close()

def hs_str(hs_list):
    hs_str=[]
    hs_list = hs_list.split(',')
    hs_list = [x.strip(' ') for x in hs_list]
    hs_list = list(filter(None, hs_list))
    for hscode in hs_list:
        if len(str(hscode))<=6:
            hs_str.append(str(hscode))
        else:
            hs_str.append(str(hscode)[:6])
    return list(set(hs_str))

def un_hscode_filter(hs_code_list):
    #宣告pool容器
    pool_df = pd.DataFrame()
    #逐一取出df['commodity_code'] == hscode
    #放入pool_df
    for hscode in hs_code_list:
        hscode = '^{}'.format(hscode)
        pool_df = pool_df.append(df[df['commodity_code'].str.contains(hscode)])
    return pool_df

def get_rank(_df):
    #按報導國GroupBy總金額
    result_df = _df.groupby('reporter_code').sum().sort_values('trade_value_us',ascending=False)

    #獲得排名
    result_df = result_df.reset_index()
    result_df['rank'] = result_df.index +1
    return result_df

def output_grid_data(_df):
    #取得比重
    _df['rate'] = _df['trade_value_us'] / sum(_df['trade_value_us'])
    #取得前三名
    grid_df = _df[:3]
    #加入台灣資料至 grid_df
    grid_df = grid_df.append(_df[_df['reporter_code']==490])
    #對應國家名稱
    grid_df = grid_df.merge(country_code,on='reporter_code')
    #
#     grid_df = grid_df[['reporter_name','rate','rank','trade_value_us']]
    grid_df = grid_df.drop_duplicates('reporter_code')
    return grid_df

def trim_output_format(industry_name,_df):
    #製作 top_3 文字
    text = ''
    for row in _df[:3].itertuples():
        text += '{}({:,.2f}%)  '.format(row.reporter_name,row.rate*100) 
    
    _output_df = _df[_df['reporter_code']==490][['rank','rate','trade_value_us']]
    _output_df['industry_name'] = industry_name
    _output_df['top_3'] = text
    return _output_df

def main_process(industry_name,hs_code_list):
    hs_code_list = hs_str(hs_code_list)
    print('{}\n{}'.format(industry_name,hs_code_list))
    main_df = un_hscode_filter(hs_code_list)
#     print(main_df)
    main_df = get_rank(main_df)
#     print(main_df.head())
    main_df = output_grid_data(main_df)
#     print(main_df.head())
    return trim_output_format(industry_name,main_df)

In [50]:
main_process('LED','85414021,85414029')

LED
['854140']


,rank,rate,trade_value_us,industry_name,top_3
1,2,0.10,"5,093,793,471.00",LED,China(33.12%) Taiwan(10.12%) Malaysia(8.72%)


In [51]:
_df = pd.DataFrame()
for industry in hs_classes.itertuples():
#     print('[{},{}]'.format(industry.industry_name,industry.hscodes))
    _df = _df.append(main_process(industry.industry_name,industry.hscodes))


紡織機械
['8445', '8449', '8447', '8446', '8448', '8444']
橡塑膠機械
['8477']
包裝機械
['8422']
食品機械
['8435', '8434', '8438']
印刷機械
['8442', '8443']
木工機械
['8465']
工具機
['8463', '8458', '8456', '8462', '8460', '8461', '8457', '8459']
航太
['8805', '880220', '8801', '880212', '8803', '880240', '880211', '880230', '8804']
電動車
['871110', '871120', '870410', '870490', '871190', '870323', '871130']
汽配
['840734', '681389', '851130', '853650', '852729', '848310', '902920', '842131', '848180', '850710', '848490', '870891', '902610', '960350', '732010', '392630', '700721', '870892', '700711', '870870', '870840', '401310', '940540', '848350', '870821', '870830', '841330', '848390', '401212', '870895', '902519', '401290', '700910', '842549', '870899', '840999', '401220', '851150', '852721', '870850', '841381', '851230', '842123', '902620', '940120', '851290', '401211', '851240', '854430', '842542', '870829', '903300', '870894', '910400', '870810', '840733', '902910', '830120', '732020', '401110', '681381', '840820

礦業
['251710', '250860', '252100', '261400', '270300', '681140', '681280', '252610', '261800', '250300', '251749', '260112', '252010', '250810', '680423', '680421', '251830', '251020', '252490', '681182', '250610', '681299', '260400', '252910', '252810', '260800', '251512', '681189', '260500', '681011', '681389', '252329', '260111', '262040', '253010', '250900', '270111', '681520', '251730', '681410', '251810', '262190', '681510', '261000', '250410', '252800', '681181', '271020', '253020', '681292', '252921', '270210', '252330', '250200', '680292', '262060', '271312', '250850', '271290', '261900', '253090', '680229', '270112', '252922', '261390', '260300', '251620', '680422', '270720', '681291', '262091', '252520', '680100', '252310', '271099', '270730', '680510', '262011', '252620', '680221', '251690', '271390', '270220', '270799', '252230', '261510', '681320', '260700', '262029', '261100', '261690', '251910', '250700', '251990', '252890', '271410', '251511', '680919', '271011', '26020

In [52]:
_df

,rank,rate,trade_value_us,industry_name,top_3
3,7,0.03,"368,049,210.00",紡織機械,China(19.87%) Germany(17.69%) Japan(15.39%)
3,6,0.05,"1,033,154,556.00",橡塑膠機械,Germany(24.21%) China(15.23%) Japan(9.51%)
3,16,0.01,"332,137,905.00",包裝機械,Germany(27.46%) Italy(22.24%) China(7.09%)
3,20,0.01,"110,759,808.00",食品機械,Germany(18.07%) Netherlands(14.91%) Italy(14...
3,20,0.01,"463,869,027.00",印刷機械,China(22.31%) Japan(11.66%) Germany(10.95%)
3,4,0.08,"521,798,253.00",木工機械,Germany(25.96%) China(20.06%) Italy(19.50%)
3,5,0.07,"2,897,583,034.00",工具機,Germany(21.14%) Japan(18.29%) Italy(8.44%)
3,23,0.00,"777,919,281.00",航太,France(26.49%) Germany(21.92%) United Kingdo...
3,23,0.00,"1,159,108,916.00",電動車,Germany(19.94%) Japan(19.67%) USA(7.99%)
3,21,0.01,"8,475,607,460.00",汽配,Germany(15.00%) China(11.81%) USA(11.33%)


In [53]:
_df.to_excel('./祐任運算_1121v3-1.xlsx')

In [267]:
_df[_df['reporter_code']==490].sort_values('commodity_code')

,reporter_code,partner_code,commodity_code,trade_value_us
195508,490,0,251511,7.656530e+05
195517,490,0,251741,3.873280e+05
196070,490,0,320649,4.109900e+07
196076,490,0,320810,1.531136e+07
196077,490,0,320820,3.301817e+07
196078,490,0,320890,6.623436e+07
196079,490,0,320910,6.823056e+06
196081,490,0,321000,4.487546e+07
196087,490,0,321410,1.735636e+07
196088,490,0,321490,6.329321e+06


In [239]:
_df[_df['reporter_code']==490]['trade_value_us'].sum()

713442671.0

In [201]:
result_df[result_df['reporter_code'] == 490]

,reporter_code,partner_code,trade_value_us,rank
17,490,0,713442671.0,18


In [137]:
##test


# for hscode in hscodelist:
#     _df = pd.DataFrame()
#     query_df = df[df.commodity_code.str.contains('^'+hscode)]
#     _df = _df.append(query_df)
#     print(_df.head())
#     fin_df = pd.DataFrame()
#     country_code.columns=['reporter_code','reporter_name']
#     result_df = _df.groupby('reporter_code').sum()
#     result_df = result_df.sort_values('trade_value_us',ascending=False).reset_index()
#     fin_df = fin_df.append(result_df[:3])
#     fin_df = fin_df.append(result_df[result_df['reporter_code'] == 490]).reset_index()
#     fin_df = fin_df.merge(country_code,on='reporter_code'); fin_df
#     fin_df['index'] = fin_df['index'] +1 

# # top_country = '\n{}\n {},{},{}\n'.format(hs_list,fin_df['reporter_name'][0],fin_df['reporter_name'][1],fin_df['reporter_name'][2])
# # top_country =top_country+str(fin_df[fin_df['reporter_code']==490][['index','trade_value_us']])
# print(fin_df)

In [138]:
for hs_grid in hs_classes['hscodes']:
    _df = pd.DataFrame()
#     print(hs_grid)
    hs_list=hs_str(hs_grid)
    print(hs_list)
    f = open("test1820.txt","a") #opens file with name of "test.txt"
    for hscode in hs_list:
        query_df = df[df.commodity_code.str.contains('^'+hscode)]
        _df = _df.append(query_df)
#         print(_df.info())
    fin_df = pd.DataFrame()
    country_code.columns=['reporter_code','reporter_name']
    result_df = _df.groupby('reporter_code').sum()
    result_df = result_df.sort_values('trade_value_us',ascending=False).reset_index()
    fin_df = fin_df.append(result_df[:3])
    fin_df = fin_df.append(result_df[result_df['reporter_code'] == 490]).reset_index()
    fin_df = fin_df.merge(country_code,on='reporter_code'); fin_df
    fin_df['index'] = fin_df['index'] +1 
    
    top_country = '\n{}\n {},{},{}\n'.format(hs_list,fin_df['reporter_name'][0],fin_df['reporter_name'][1],fin_df['reporter_name'][2])
    top_country =top_country+str(fin_df[fin_df['reporter_code']==490][['index','trade_value_us']])
    f.write(top_country)
    f.close()
    

['8445', '8447', '8449', '8448', '8446', '8444']
['8477']
['8422']
['8434', '8435', '8438']
['8443', '8442']
['8465']
['8461', '8458', '8456', '8463', '8457', '8462', '8460', '8459']
['8803', '8805', '880212', '880240', '8804', '880220', '880230', '8801', '880211']
['871130', '871120', '870490', '870410', '871110', '871190', '870323']
['870850', '401211', '851240', '848350', '851140', '870821', '848490', '851120', '902620', '870895', '401110', '842131', '700721', '848390', '851190', '401220', '840999', '841330', '830230', '851110', '851290', '870710', '870810', '870892', '700711', '731811', '830120', '841381', '848310', '870790', '850710', '848180', '910400', '870829', '853650', '401120', '732010', '840733', '681389', '870830', '870840', '902610', '732020', '851150', '902920', '870899', '681320', '851130', '842123', '851230', '940540', '401290', '401310', '841520', '854430', '392630', '842549', '903300', '840734', '840991', '851220', '902519', '852729', '940120', '870891', '681381', '8

['330710', '330430', '330491', '330530', '340120', '330720', '340111', '330730', '330590', '330510', '330610', '330790', '330499', '330300', '330520', '330410', '340130', '330420']
['621111', '950640', '950790', '950662', '621120', '950661', '630640', '620293', '630699', '950629', '620193', '950611', '950670', '640219', '890310', '950710', '950720', '620199', '950612', '950632', '950730', '620113', '640411', '950699', '950420', '630720', '630691', '950490', '940430', '640212', '950619', '950669', '640312', '620192', '950659', '900490', '621112', '610130', '950691', '900410', '640319', '890790', '950621', '630630', '950631', '610230', '950639', '901910', '420321', '950651', '630622']
['401320', '871499', '871494', '871495', '871491', '871492', '731511', '851210', '871200', '871496', '871190', '401150', '871493']
['50', '62', '61', '52', '53', '59', '55', '54', '63', '51', '58', '60', '57', '56']
['9402', '9403', '9401', '9404']
['940520']
['950430', '640299', '611130', '950490', '950410

In [89]:

fin_df

,index,reporter_code,partner_code,trade_value_us,reporter_name
0,1,156,0,3.758383e+12,China
1,2,842,0,2.324422e+12,USA
2,3,276,0,2.316974e+12,Germany
3,15,490,0,5.511983e+11,"Other Asia, nes"
